In [0]:
create or refresh streaming table bronze_customers
comment "This is the bronze table"
TBLproperties("quality" = "bronze")
as
select * ,
_metadata.file_path as input_file_path,
current_timestamp as ingestion_timestamp 
from 
cloud_files
('/Volumes/circuitbox/landing/operationaldata/customers/*.json',
'json',
map('cloudFiles.inferColumnTypes','true')
);

In [0]:
create or refresh streaming table silver_customers_clean(
  constraint valid_customerid expect (customer_id is not null) on violation fail update,
  constraint valid_customername expect (customer_name is not null) on violation drop row,
  constraint valid_telephone expect (length(telephone)>=10),
  constraint valid_email expect (email is not null),
  constraint valid_dateofbirth expect (date_of_birth >= '1920-01-01')
)
comment "This table contains clean data from bronze"
tblproperties ("quality" = "true")
as
select
cast(created_date as date) as created_date,
customer_id,
customer_name,
cast(date_of_birth as date) as date_of_birth,
email,
telephone
from STREAM(LIVE.bronze_customers)